In [1]:
# Se descarga en el directorio actual el documento 'minsearch.py' de la ruta indicada
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-04 19:06:56--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-04 19:06:56 (47.2 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [10]:
# libreria generada en video premodule1
import minsearch

In [11]:
# se importa documents.json, contiene documentos por cada curso(data-engieering, ml, etc) que incluye text, section y question.
import json

with open('documents.json') as documentos_curso:
    documents = json.load(documentos_curso)

In [12]:
print(f'Tamaño: {len(documents)}')
print(f'Tipo: {type(documents)}\n')

for i in range(len(documents)):
    print(documents[i]['course'] )


Tamaño: 3
Tipo: <class 'list'>

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp


In [13]:
# primer elemento de documents del curso 'data-engineering-zoomcamp'
documents[0]['documents'][0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?'}

In [4]:
# print(documents[0])

In [15]:
# Estructura
'''
Ejemplo del primer elemento de document. (Contiene 3) 
>> documents[0]  (diccionario)
{
'course': string 
        'data-engineering-zoomcamp'
'documents': list of dictionaries 
        [ {text: '', section: '' , question: ''},
          {text: '', section: '' , question: ''},
                         ... 
          {text: '', section: '' , question: ''} ] ,
          ...
}
'''
# Lista para almacenar el conjunto total de documents. 
# Tipo: Lista de diccionarios
# Se va a utilizar para indexar
docs=[]

for i in documents:
    for elemento in i['documents']:
        elemento['course'] = i['course'] # agregamos esta columna
        docs.append(elemento) 
    

In [7]:
docs[15]
# documentos para indexar

{'text': 'No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y\nOlder news:[source1] [source2]',
 'section': 'General course-related questions',
 'question': 'Homework - Are late submissions of homework allowed?',
 'course': 'data-engineering-zoomcamp'}

In [17]:
'''
Usamos de minsearch.py la clase Index para crear el objeto 'index'.
Se inicializa con los atributos de text_fields, keyword_fields
Luego aplicamos la función fit con los documentos preparados (docs)

NOTA: revisar a mayor profundidad el documento 'minsearch.py'
'''

"\nUsamos de minsearch.py la clase Index para crear el objeto 'index'.\nSe inicializa con los atributos de text_fields, keyword_fields\n"

In [8]:
# indexamos los documentos con minsearch
index= minsearch.Index(
    text_fields =['question','text','section'],  #for performance search
    keyword_fields=['course'],
        
)

index.fit(docs)

In [ ]:
'''
Función search, retorna lista de diccionarios (documentos) donde cada elemento coincide con el criterio de búsqueda.
Parámetros dados: query, diccionario de pesos, diccionario de filtros, número de resultados 
'''

In [10]:
#query
pregunta= 'the course has already started. Can I still joint it?'
pesos_diccionario= {'text':0.2, 'section':0.5,'question':4.0}
curso_filtro = {'course':'data-engineering-zoomcamp'}

resultados= index.search(
    query= pregunta ,
    boost_dict= pesos_diccionario,
    filter_dict= curso_filtro, 
    num_results= 5
)

resultados


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [17]:
print(type(resultados))

<class 'list'>


In [11]:
'''
Se necesita construir un formato de CONTEXT para ser agregado a prompt. 
Se ordena los elementos de 'resultados'.

Output esperado (string):
"{section:'', question: '', text:'' } ,... , {} "
'''
contexto= ""
for i in resultados:
    contexto = contexto + f"Section: {i['section']} \nQuestion: {i['question']} \nText: {i['text']} \n\n" 

# resultados[0]['text']
print(contexto)


Section: General course-related questions 
Question: Course - Can I still join the course after the start date? 
Text: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 

Section: General course-related questions 
Question: Course - Can I follow the course after it finishes? 
Text: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project. 

Section: General course-related questions 
Question: Course - What can I do before the course starts? 
Text: You can start by installing and setting up all the dependencies and requirements:
Google cloud account
Google Cloud SDK
Python 3 (installed with Anaconda)
Terra

In [12]:
# q
pregunta

'the course has already started. Can I still joint it?'

In [ ]:
'''
Juntamos todo en el prompt y formateamos con la query(pregunta) y el context(base de datos)
'''

In [13]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION from FAQ 
database.
If the CONTEXT doesn't contain the answer, output NONE.
ALWAYS finish with kind words, Example: ask if need anything else or need more information.

QUESTION: {question}

CONTEXT: 
{context} 

"""
# strip: elimina espacios en blanco, al inicio y al final
prompt_final= prompt_template.format(question= pregunta, context= contexto).strip() 


In [14]:
print(prompt_final)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION from FAQ 
database.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started. Can I still joint it?

CONTEXT: 
Section: General course-related questions 
Question: Course - Can I still join the course after the start date? 
Text: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 

Section: General course-related questions 
Question: Course - Can I follow the course after it finishes? 
Text: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your 

In [ ]:
'''
 Insertamos el prompt en nuestro modelo LLM
'''

In [2]:
import os
api_key= os.environ.get('OPENAI_API_KEY') # verifica que el api_key esté configurado

In [3]:
import openai
from openai import OpenAI

client = OpenAI()

In [4]:
client

In [19]:
response = client.chat.completions.create(
    model ='gpt-4o-mini',
    messages = [
        {
            "role":"user",
            "content": pregunta
        }
    ]
)

In [26]:
# respuesta sin contexto 
response.choices[0].message.content

'It depends on the policies of the institution or organization offering the course. Many courses allow late enrollments, while others may have strict deadlines. I recommend reaching out to the course instructor or administration to inquire about the possibility of joining the course, even if it has already started. They can provide you with the best information regarding availability and any necessary steps you need to take.'

In [ ]:
#RESPUESTA CON EL PROMPT CON EL CONTEXTO 

In [36]:
response = client.chat.completions.create(
    model ='gpt-4o-mini',
    messages = [
        {
            "role":"user",
            "content": prompt_final
        }
    ]
)

In [37]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [41]:
# respuesta en español
prompt_final=prompt_template.format(question='El curso ya comenzó, puedo unirme todavía?', context= contexto).strip()

response = client.chat.completions.create(
    model ='gpt-4o-mini',
    messages = [
        {
            "role":"user",
            "content": prompt_final
        }
    ]
)

response.choices[0].message.content

'Sí, puedes unirte al curso aunque ya haya comenzado. Aún eres elegible para enviar las tareas, pero ten en cuenta que habrá fechas límite para entregar los proyectos finales.'

In [ ]:
'''
Podemos modularizar, crear el flujo por funciones
- Busqueda por index
- Ordenamiento de los documentos y generación del prompt
- Generación delouput por el llm
'''

In [35]:
# FUNCION: busqueda por index
''' courses:
data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp
'''
def busqueda(pregunta,curso, num_results ):
    # por defecto
    pesos_diccionario= {'text':0.2, 'section':0.5,'question':4.0}
    
    resultados= index.search(
    query= pregunta ,
    boost_dict= pesos_diccionario,
    filter_dict= {'course':curso}, 
    num_results= num_results
    )

    return resultados

In [39]:
pregunta= 'is importan to know about math?'
curso= 'machine-learning-zoomcamp'

resultados = busqueda(pregunta, curso, 3)
resultados

[{'text': "Yes! We'll cover some linear algebra in the course, but in general, there will be very few formulas, mostly code.\nHere are some interesting videos covering linear algebra that you can already watch: ML Zoomcamp 1.8 - Linear Algebra Refresher from Alexey Grigorev or the excellent playlist from 3Blue1Brown Vectors | Chapter 1, Essence of linear algebra. Never hesitate to ask the community for help if you have any question.\n(Mélanie Fouesnard)",
  'section': 'General course-related questions',
  'question': "I don't know math. Can I take the course?",
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Check this article. If you know everything in this article, you know enough. If you don’t, read the article and join the coursIntroduction to Pythone too :)\nIntroduction to Python – Machine Learning Bookcamp\nYou can follow this English course from the OpenClassrooms e-learning platform, which is free and covers the python basics for data analysis: Learn Python Basics for Dat

In [19]:
# FUNCION: ordena los resultados(section, question, text) y completa el prompt final
def build_prompt(pregunta, resultados):
    contexto= ""
    for i in resultados:
        contexto= contexto + f"Section: {i['section']} \nQuestion: {i['question']} \nText: {i['text']} \n\n"
    # contexto con los documento en orden
        
    prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION from FAQ 
database.
If the CONTEXT doesn't contain the answer, output NONE.
Always finish you message with questions if need anything else or similar.
    
QUESTION: {question}
    
CONTEXT: 
{context}    
    """
    prompt_final= prompt_template.format(question= pregunta, context= contexto).strip() 
    
    return prompt_final


In [59]:
prompt_final= build_prompt(pregunta, resultados)
print(prompt_final)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when answering the QUESTION from FAQ 
database.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: is importan to know about math?

CONTEXT: 
Section: General course-related questions 
Question: I don't know math. Can I take the course? 
Text: Yes! We'll cover some linear algebra in the course, but in general, there will be very few formulas, mostly code.
Here are some interesting videos covering linear algebra that you can already watch: ML Zoomcamp 1.8 - Linear Algebra Refresher from Alexey Grigorev or the excellent playlist from 3Blue1Brown Vectors | Chapter 1, Essence of linear algebra. Never hesitate to ask the community for help if you have any question.
(Mélanie Fouesnard) 

Section: General course-related questions 
Question: How much Python should I know? 
Text: Check this article. If you know everything in this article, you know enough. If you don

In [20]:
# Funcion del modelo

def chat_llm(prompt_final):
    
    response= client.chat.completions.create(
        model= 'gpt-4o-mini', 
        messages= [
            {
                'role':'user',
                'content': prompt_final
            }
        ]
    )

    return response.choices[0].message.content

    

In [24]:
print(chat_llm(prompt_final))

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


In [91]:
# Entonces el flujo final:
pregunta= 'how can i install terraform?'
curso= 'data-engineering-zoomcamp'

fuente_datos = busqueda(pregunta, curso, 3)
prompt_final= build_prompt(pregunta, fuente_datos)
output_llm= chat_llm(prompt_final)


In [92]:
print(output_llm)

To install Terraform for WSL, you can refer to the following link: [Configuring Terraform on Windows 10 Linux Subsystem](https://techcommunity.microsoft.com/t5/azure-developer-community-blog/configuring-terraform-on-windows-10-linux-sub-system/ba-p/393845). 

If you need anything else, feel free to ask!


In [ ]:
'''
Tambien podemos modularizar lo último: función completa de RAG
'''

In [94]:
def rag(query, course):
    fuente_datos = busqueda(pregunta, curso, 5)
    prompt_final= build_prompt(pregunta, fuente_datos)
    output_llm= chat_llm(prompt_final)

    return output_llm

In [95]:
pregunta= 'how do i run kafka?'
curso= 'data-engineering-zoomcamp'

rag(pregunta, curso)

"To run Kafka, based on the context provided, you can execute the following command in the project directory for Java Kafka:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you're using Python, make sure to create a virtual environment and run the necessary Python files within that environment. Do you need help with anything else?"

In [ ]:
'''
Ahora se usa en elastic_search
'''

In [2]:
# en bash: pip install elasticsearch==8.4.3
import elasticsearch
print(elasticsearch.__version__)

(8, 4, 3)


In [8]:
from elasticsearch import Elasticsearch

# elastic search client
es_client= Elasticsearch('http://127.0.0.1:9200/')
print(es_client.info())

{'name': '4a7d2faec117', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BukzAOOjTrezqaJlv5cuWQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [9]:
! curl http://localhost:9200

{
  "name" : "4a7d2faec117",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "BukzAOOjTrezqaJlv5cuWQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
'''
Creamos el index con ElasticSearch
'''

In [11]:
# Configuracion
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'
es_client.indices.create(index= index_name, body= index_settings)


In [18]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# Documentos para indexar
docs[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
# indexamos cada uno de los documentos 
for i in tqdm(docs):
    es_client.index(index=index_name, document=i)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:04<00:00, 205.41it/s]


In [12]:
query= 'the course has already started. Can I still joint it?'
course= "data-engineering-zoomcamp"

In [13]:
search_query= {
    "size": 5,
    
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": course
                }
            }
        }
    }
}

In [14]:
# busqueda
respuesta_search= es_client.search(index= index_name, body= search_query)

/tmp/ipykernel_84707/2880328538.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  respuesta_search= es_client.search(index= index_name, body= search_query)


In [15]:
respuesta_search['hits']['hits'][0]['_source']

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [16]:
resultados_elasticsearch =[]

for i in respuesta_search['hits']['hits']:
    resultados_elasticsearch.append(i['_source'])

resultados_elasticsearch

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [ ]:
'''
De igual manera se puede modularizar
'''

In [17]:
# Funcion de elastic_search para obtención de bd
def elastic_search(query, course):
    search_query= {
        "size": 5,
        
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    # busqueda
    respuesta_search= es_client.search(index= index_name, body= search_query)
    
    resultados_elasticsearch =[]

    for i in respuesta_search['hits']['hits']:
        resultados_elasticsearch.append(i['_source'])
    
    return resultados_elasticsearch


In [18]:
query= 'the course has already started. Can I still joint it?'
course= "data-engineering-zoomcamp"

elastic_search(query, course)

/tmp/ipykernel_84707/1494026147.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  respuesta_search= es_client.search(index= index_name, body= search_query)


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta

In [ ]:
'''
De la función previa RAG definida
'''

In [23]:
def rag(query, course):
    fuente_datos = elastic_search(query, course)
    prompt_final= build_prompt(query, fuente_datos)
    output_llm= chat_llm(prompt_final)

    return output_llm

In [24]:
rag(query, course)

/tmp/ipykernel_84707/1494026147.py:25: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  respuesta_search= es_client.search(index= index_name, body= search_query)


"Yes, you can still join the course after the start date. Even if you don't register, you're still eligible to submit the homeworks. Just be mindful of the deadlines for the final projects and try not to leave everything until the last minute. \n\nDo you need any more information or help with something else?"